In [1]:
!pip install langchain_community
!pip install -qU pypdf
!pip install transformers torch
!pip install faiss-cpu
!pip install faiss-gpu

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.4/50.4 kB 888.8 kB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.4/2.4 MB 27.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 32.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 404.4/404.4 kB 15.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 295.8/295.8 kB 14.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.4/76.4 kB 4.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 78.0/78.0 kB 4.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.3/49.3 kB 2.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 141.9/141.9 kB 6.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 54.5/54.5 kB 2.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 2.3 MB/s eta 0:00:00
  Attempting uninstall: tenacity
    Found existing installation: tenacity 9.0.0
    Uninstalling tenacity-9.0.0:
      Successfully

In [2]:
from langchain_community.document_loaders import PyPDFLoader
import re

from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_community.document_loaders import TextLoader

from transformers import AutoTokenizer, AutoModel
import torch

import numpy as np
import faiss

from langchain.vectorstores import FAISS
from langchain.embeddings import HuggingFaceEmbeddings

# DataLoader

In [3]:
# Загрузка PDF-файла с помощью PyPDFLoader
pdf_path = "/content/sample_data/test1.pdf"
loader = PyPDFLoader(pdf_path)

# Загрузка текста из PDF
pages = loader.load()  # загружаем весь текст, без разделения

# Преобразование списка страниц в один большой текст
full_text = ''.join([page.page_content for page in pages])

# Разделение текста по слову "\nРаздел ", сохраняя слово в результатах
split_text = full_text.split("\nРаздел ")

# Добавляем "Раздел" обратно к каждому фрагменту, кроме первого
split_text = [f"Раздел {fragment.strip()}" for fragment in split_text if fragment.strip()]

# Выводим результат и сохраняем в текстовые файлы
for i, fragment in enumerate(split_text):

    cleaned_fragment = fragment

    # Формируем имя файла
    file_name = f"{i}.txt"

    # Сохраняем фрагмент в текстовый файл
    with open(file_name, 'w', encoding='utf-8') as file:
        file.write(cleaned_fragment.strip() + '\n')

    # Выводим сообщение о сохранении
    print(f"Сохранен: {file_name}")

Сохранен: 0.txt
Сохранен: 1.txt
Сохранен: 2.txt
Сохранен: 3.txt
Сохранен: 4.txt
Сохранен: 5.txt
Сохранен: 6.txt
Сохранен: 7.txt
Сохранен: 8.txt
Сохранен: 9.txt
Сохранен: 10.txt
Сохранен: 11.txt
Сохранен: 12.txt


# Splitter

In [4]:
def load_and_split_text(file_path):
    # Настройка разделения текста на параграфы
    text_splitter = RecursiveCharacterTextSplitter(
        chunk_size=200, chunk_overlap=50, add_start_index=True
    )

    loader = TextLoader(file_path) # Создаем загрузчик текстовых данных
    docs = loader.load() # Загружаем содержимое файла

    all_splits = text_splitter.split_documents(docs)# Разделяем документ на параграфы
    texts = [doc.page_content for doc in all_splits] # Извлекаем содержимое параграфов
    return texts

Пример использования функции, переменная texts содержит массив параграфов из указанного файла

In [5]:
file_path = '/content/10.txt'
texts = load_and_split_text(file_path)

for i, paragraph in enumerate(texts):
    print(f"Параграф {i + 1}:")
    print(paragraph)
    print('-' * 50)  # Разделитель между параграфами для удобства чтения

Параграф 1:
Раздел 10. Обязательства Работнико в 
10.1. Соблюдать Правила внутреннего трудового распорядка, выполнять 
возложенные трудовые обязанности в соответствии с трудовым договором и
--------------------------------------------------
Параграф 2:
должностной инструкцией, качественно и ответственно выполнять 
производственные задания.  
10.2. Способствовать внедре нию инноваций, постоянно повышать свою
--------------------------------------------------
Параграф 3:
квалификацию, в том числе путем самообразования.  
10.3. Обеспечивать непрерывность перевозочного процесса, соблюдать 
требования безопасности движения поездов, охраны труда, предусмотренные
--------------------------------------------------
Параграф 4:
нормативными правовыми актами  Российской Федерации и локальными 
нормативными актами Работодателя, выполнять предусмотренные системой 
корпоративного медицинского обслуживания профилактические и
--------------------------------------------------
Параграф 5:
оздоровительн

Создадим all_texts, в котором будеь лежать texts для каждого файла

In [25]:
all_texts = []
for i in range(3):
    file_path = f"/content/{i}.txt"  # Путь к файлу
    texts = load_and_split_text(file_path)  # Загружаем и разбиваем текст
    all_texts.append(texts)  # Добавляем массив texts в all_texts

Пример вывода все параграфов

In [26]:
# Проходим по каждому файлу в all_texts
for i, texts in enumerate(all_texts):
    print(f"File {i}:")

    # Проходим по каждому параграфу в файле
    for j, text in enumerate(texts):
        # Выводим первые 200 символов каждого параграфа
        print(f"  Paragraph {j}: {text[:200]}...")
    print()  # Добавляем пустую строку для разделения выводов между файлами

File 0:
  Paragraph 0: Раздел 2...

File 1:
  Paragraph 0: Раздел 1. Основные понятия  
Коллективный договор ОАО «РЖД» на 2023 - 2025 годы - правовой акт, 
регулирующий социально -трудовые отношения в открытом акционерном...
  Paragraph 1: обществе «Российские железные дороги»  между сторонами социального 
партнерства - Работниками и Работодателем в лице их представителей.  
Настоящий Договор является единым для ОАО «РЖД», включая...
  Paragraph 2: филиалы, структурные подразделения и представительства, за исключением 
Петропавловского отделения Южно -Уральской железной дороги - филиала...
  Paragraph 3: ОАО «РЖД», расположенного на территории Республики  Казахстан, в котором, 
на основе настоящего Договора и с учетом особенностей законодательства...
  Paragraph 4: Республики Казахстан, заключается отдельный коллективный договор на 2023 - 
2025 годы.  
Предоставление Работникам одних и тех же гарантий, льгот и...
  Paragraph 5: компенсаций в соответствии с Договором и коллективным дого

# Функция для создания эмбеддингов с помощью Hugging Face

In [8]:
def get_embedding(text_list):
    # Загрузка модели и токенизатора
    model_name = 'intfloat/multilingual-e5-large'
    tokenizer = AutoTokenizer.from_pretrained(model_name)
    model = AutoModel.from_pretrained(model_name)

    inputs = tokenizer(text_list, return_tensors='pt', padding=True, truncation=True) # Подготовка текста
    with torch.no_grad(): # Получение эмбеддингов
        outputs = model(**inputs)
        embeddings = outputs.last_hidden_state  # Или используйте outputs.pooler_output для токена [CLS]
    # Получение эмбеддинга для всего текста
    text_embedding = torch.mean(embeddings, dim=1)  # Среднее значение по всем токенам

    return text_embedding  # Возвращаем массив эмбеддингов

Пример использования

In [9]:
text_list = [
    "Hello, how are you?",
    "I am learning about embeddings.",
    "This is a test sentence."
]
embedding = get_embedding(text_list) # Получение эмбеддингов
print("Embeddings shape:", embedding.shape)
print("Embeddings:", embedding)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/418 [00:00<?, ?B/s]

sentencepiece.bpe.model:   0%|          | 0.00/5.07M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/17.1M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/280 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/690 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/2.24G [00:00<?, ?B/s]

Embeddings shape: torch.Size([3, 1024])
Embeddings: tensor([[ 1.2003, -0.2312, -0.9946,  ..., -0.3647, -0.9960,  0.4997],
        [ 0.0892,  0.1687, -0.3615,  ..., -0.4328, -0.8738,  0.3127],
        [ 0.0087, -0.1989, -0.8174,  ..., -0.3237, -0.7941,  0.8018]])


In [10]:
def get_embeddings(texts):
    embeddings_list = [] # Список для хранения эмбеддингов
    for paragraph in texts: # Перебор каждого параграфа
        text_embedding = get_embedding(paragraph)
        embeddings_list.append(text_embedding)
    return embeddings_list  # Возвращаем массив эмбеддингов

In [11]:
# Переменная для хранения эмбеддингов всех файлов
all_embeddings = []

# Проходим по каждому списку параграфов (по каждому файлу)
for i, texts in enumerate(all_texts):
    print(f"Processing file {i}...")
    embeddings_list = get_embeddings(texts) # Получаем эмбеддинги для параграфов данного файла
    embeddings_tensor = torch.cat(embeddings_list).numpy().astype('float32') # Преобразуем список эмбеддингов в тензор и преобразуем в numpy массив и изменяем тип на float32
    all_embeddings.append(embeddings_tensor) # Добавляем эмбеддинги этого файла в общий список

Processing file 0...
Processing file 1...
Processing file 2...


In [30]:
all_embeddings[2]

array([[ 0.6259444 , -0.505826  , -0.98722005, ...,  0.1490781 ,
        -1.1312962 , -0.04787919],
       [ 1.3277814 , -0.62727875, -1.1346494 , ...,  0.30252212,
        -0.3525623 ,  0.30110142],
       [ 0.1563436 , -0.4872395 , -0.5220888 , ..., -0.10359735,
        -1.15211   ,  0.18427394],
       ...,
       [ 0.2697898 , -0.8724825 , -0.7117272 , ...,  0.00588362,
        -0.460569  , -0.05630865],
       [-0.24618994, -0.6202959 , -0.5034256 , ..., -0.37022296,
        -0.8908498 ,  0.28156686],
       [ 0.5553218 ,  0.44651487, -0.6289744 , ...,  0.17343809,
        -1.6786376 , -0.21959712]], dtype=float32)

In [31]:
for i, embeddings in enumerate(all_embeddings):
    print(f"Embeddings for file {i}:")
    print(embeddings)  # Выводим все эмбеддинги файла
    print()


Embeddings for file 0:
[[ 0.56014824  0.2769879  -0.26688212 ...  0.28067023 -1.0268214
   0.3975392 ]]

Embeddings for file 1:
[[ 0.25743118  0.06029226 -0.5172514  ...  0.15048915 -0.7528636
  -0.15051259]
 [ 0.27518904 -0.84552073 -0.39267606 ...  0.04381828 -0.9728229
   0.23378725]
 [ 0.46307224 -0.9974735  -0.14786763 ... -0.46558774 -1.7377483
   0.512345  ]
 ...
 [ 0.29797307 -0.4427656  -0.8981262  ... -0.8758098  -0.8974271
  -0.1290108 ]
 [ 0.3352763  -1.1462281  -0.8996349  ... -0.4933166  -0.9600326
   0.46321502]
 [ 0.21181777 -0.69002074 -0.56313807 ... -0.00860083 -0.88163304
   0.16861922]]

Embeddings for file 2:
[[ 0.6259444  -0.505826   -0.98722005 ...  0.1490781  -1.1312962
  -0.04787919]
 [ 1.3277814  -0.62727875 -1.1346494  ...  0.30252212 -0.3525623
   0.30110142]
 [ 0.1563436  -0.4872395  -0.5220888  ... -0.10359735 -1.15211
   0.18427394]
 ...
 [ 0.2697898  -0.8724825  -0.7117272  ...  0.00588362 -0.460569
  -0.05630865]
 [-0.24618994 -0.6202959  -0.5034256  .

In [46]:
len(all_embeddings[2])

24

In [42]:
len(texts)

24

In [47]:
emb = all_embeddings[2]
text_list = all_texts[2]

In [50]:
len(emb)

24

In [49]:
print(emb)

[[ 0.6259444  -0.505826   -0.98722005 ...  0.1490781  -1.1312962
  -0.04787919]
 [ 1.3277814  -0.62727875 -1.1346494  ...  0.30252212 -0.3525623
   0.30110142]
 [ 0.1563436  -0.4872395  -0.5220888  ... -0.10359735 -1.15211
   0.18427394]
 ...
 [ 0.2697898  -0.8724825  -0.7117272  ...  0.00588362 -0.460569
  -0.05630865]
 [-0.24618994 -0.6202959  -0.5034256  ... -0.37022296 -0.8908498
   0.28156686]
 [ 0.5553218   0.44651487 -0.6289744  ...  0.17343809 -1.6786376
  -0.21959712]]


In [51]:
len(text_list)

24

In [52]:
print(text_list)

['Раздел 2. Общие положения  \n2.1. Настоящий Договор заключен между Работниками и Работодателем в \nлице их представителей (далее - Стороны) на добровольной и равноправной \nоснове в целях:', 'основе в целях:  \nсоздания системы социально -трудовых отношений, максимально \nспособствующей стабильной и эффективной деятельности, повышению', 'материального и социального обеспечения Работников, укреплению деловой \nрепутации;  \nусиления социальной ответственности Сторон з а результаты \nпроизводственно -экономической деятельности;', 'производственно -экономической деятельности;  \nсоздания условий, способствующих повышению безопасности труда;  \nобеспечения роста мотивации и производительности труда Работников за', 'счет предоставления предусмотренных настоящим Договором социальных \nгарантий, льгот и компенсаций, а также роста благосостояния и уровня', 'социальной защиты Работников, их семей, неработающих пенсионеров и лиц, \nприравненных к неработающим пенсионерам;  \nсоздания благоприя

# Milvus

In [15]:
!pip install -U pymilvus

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 198.9/198.9 kB 6.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.4/49.4 MB 9.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 53.6/53.6 kB 3.7 MB/s eta 0:00:00


In [16]:
from pymilvus import MilvusClient

client = MilvusClient("milvus_demo.db")

DEBUG:pymilvus.milvus_client.milvus_client:Created new connection using: d80da954b019400c917fc7ed8cf386b1


In [17]:
if client.has_collection(collection_name="demo_collection"):
    client.drop_collection(collection_name="demo_collection")
client.create_collection(
    collection_name="demo_collection",
    dimension = 1024,  # The vectors we will use in this demo has 768 dimensions
)

DEBUG:pymilvus.milvus_client.milvus_client:Successfully created collection: demo_collection
DEBUG:pymilvus.milvus_client.milvus_client:Successfully created an index on collection: demo_collection


In [53]:
# Each entity has id, vector representation, raw text, and a subject label that we use
# to demo metadata filtering later.
emb = all_embeddings[2]
text_list = all_texts[2]

data = [
    {"id": i, "vector": all_embeddings[2], "text": texts[2], "subject": "history"}
    # for i in range(len(all_embeddings))
]

print("Data has", len(data), "entities, each with fields: ", data[0].keys())
print("Vector dim:", len(data[0]["vector"]))

Data has 1 entities, each with fields:  dict_keys(['id', 'vector', 'text', 'subject'])
Vector dim: 24


In [57]:
query_vectors = get_embedding(["Who is Alan Turing?"])
# If you don't have the embedding function you can use a fake vector to finish the demo:
# query_vectors = [ [ random.uniform(-1, 1) for _ in range(768) ] ]

res = client.search(
    collection_name="demo_collection",  # target collection
    data=query_vectors,  # query vectors
    limit=2,  # number of returned entities
    output_fields=["text", "subject"],  # specifies fields to be returned
)

print(res)

ERROR:pymilvus.decorators:RPC error: [search], <ParamError: (code=1, message=`search_data` value tensor([[ 0.8419,  0.4270, -0.0197,  ..., -0.4834,  0.0196, -0.2170]]) is illegal)>, <Time:{'RPC start': '2024-10-12 12:33:06.860621', 'RPC error': '2024-10-12 12:33:07.260005'}>
ERROR:pymilvus.milvus_client.milvus_client:Failed to search collection: demo_collection


ParamError: <ParamError: (code=1, message=`search_data` value tensor([[ 0.8419,  0.4270, -0.0197,  ..., -0.4834,  0.0196, -0.2170]]) is illegal)>